# Setup the environment

In [ ]:
import scanpy as sc
import pandas as pd
import numpy as np
import seaborn as sb
from scipy.stats import median_abs_deviation
import scipy.sparse as sp
import matplotlib.pyplot as plt
import anndata
import scvi
import celltypist
from celltypist import models
import scrublet as scb
import palantir

# Load the data

In [ ]:
data_path = 'D:/Python/Data/GSE208653_RAW/' # Set the folder location

In [ ]:
adata=sc.read_mtx(data_path + 'GSM6360680_N_HPV_NEG_1.matrix.mtx.gz') # Main data
adata_barcodes=pd.read_csv(data_path + 'GSM6360680_N_HPV_NEG_1.barcodes.tsv.gz', header=None) # Loads unique identifiers for each cell
adata_features=pd.read_csv(data_path + 'GSM6360680_N_HPV_NEG_1.features.tsv.gz', header=None, sep='\t') # Loads gene names and information
adata=adata.T # Transpose the matrix (Flips the data orientation)

# Assign metadata to AnnData object
adata.var_names = adata_features.iloc[:, 1].values  # Assigns gene names to columns (var = variables/genes)
adata.obs_names = adata_barcodes.iloc[:, 0].values  # Assigns cell barcodes to rows (obs = observations/cells)
adata.var['gene_id'] = adata_features.iloc[:, 0].values  #  Stores Ensembl gene IDs (scientific gene identifiers)
adata.var['feature_types'] = adata_features.iloc[:, 2].values  # Stores gene types (protein-coding, non-coding, etc.)


In [ ]:
adata.var_names_make_unique() 

This ensures no duplicate gene names, Some genes might have identical names, this adds suffixes like "_1", "_2"

In [ ]:
adata.var

In [ ]:
adata.obs

# QC and Filtering

## Quality control

In [ ]:
# mitochondrial genes
adata.var["mt"] = adata.var_names.str.startswith("MT-")
# ribosomal genes
adata.var["ribo"] = adata.var_names.str.startswith(("RPS", "RPL"))
# hemoglobin genes.
adata.var["hb"] = adata.var_names.str.contains("^HB[^(P)]")

In [ ]:
sc.pp.calculate_qc_metrics(
    adata, qc_vars=["mt", "ribo", "hb"], inplace=True, percent_top=[20], log1p=True
)
adata

In [ ]:
# Create comprehensive QC plots
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Total counts distribution
sc.pl.violin(adata, 'total_counts', ax=axes[0,0])
axes[0,0].set_title('Total Counts per Cell')

# Mitochondrial gene percentage
sc.pl.violin(adata, 'pct_counts_mt', ax=axes[0,1]) 
axes[0,1].set_title('Mitochondrial Gene %')

# Genes per cell vs total counts
sc.pl.scatter(adata, 'total_counts', 'n_genes_by_counts', 
              color='pct_counts_mt', ax=axes[1,0])
axes[1,0].set_title('Genes vs Counts (colored by MT%)')

# Gene expression distribution
sc.pl.highest_expr_genes(adata, n_top=20, ax=axes[1,1])
axes[1,1].set_title('Top 20 Expressed Genes')

plt.tight_layout()
plt.show()

## Filtering

In [ ]:
sc.pp.filter_cells(adata, min_genes=500)  # Remove cells with fewer than 500 genes
sc.pp.filter_cells(adata, min_counts=2000)  # Remove cells with fewer than 2000 total RNA counts
adata = adata[adata.obs["pct_counts_mt"] < 10, :]  # Remove cells with more than 10% mitochondrial content
sc.pp.filter_genes(adata, min_cells=10) # Filter genes expressed in fewer than 10 cells


### Doublet filtering

In [ ]:
# filtering/preprocessing parameters:
min_counts = 2
min_cells = 3
vscore_percentile = 85
n_pc = 50

# doublet detector parameters:
expected_doublet_rate = 0.02
sim_doublet_ratio = 3
n_neighbors = 15


scrub = scb.Scrublet(
    counts_matrix=adata.X,
    n_neighbors=n_neighbors,
    sim_doublet_ratio=sim_doublet_ratio,
    expected_doublet_rate=expected_doublet_rate,
)

doublet_scores, predicted_doublets = scrub.scrub_doublets(
    min_counts=min_counts,
    min_cells=min_cells,
    n_prin_comps=n_pc,
    use_approx_neighbors=True,
    get_doublet_neighbor_parents=False,
)

adata.obs["doublet_score"] = doublet_scores
adata.obs["doublet"] = predicted_doublets

**Code Breakdown**


In [ ]:
sb.histplot(adata.obs["doublet_score"], kde=True, bins=50)
plt.xlabel("Doublet Score")
plt.ylabel("Density")
plt.title("Histogram of Doublet Scores")
plt.show()



sc.pl.violin(
    adata,
    keys=["doublet_score"],  
    jitter=0.4,  # Adds scatter points for better visualization
    rotation=45,  # Rotates x-axis labels for readability
    show=True
)

In [ ]:
#thr = #add threshold
#ix_filt = adata.obs['doublet_score']<=thr

#adata = adata[ix_filt].copy()
#print('Number of cells after doublet filter: {:d}'.format(adata.n_obs))

# Pre-processing

## Normalization

In [ ]:
adata.layers["raw_data"] = adata.X.copy()

In [ ]:
sc.pp.normalize_total(adata, inplace=True)
sc.pp.log1p(adata)

In [ ]:
#adata.write_h5ad(data_path + "processed_data.h5ad")

## Highly Variable genes

In [ ]:
sc.pp.highly_variable_genes(
    adata, layer="raw_data", n_top_genes=4000, flavor="seurat_v3"
)

In [ ]:
adata.var[
    ["highly_variable", "highly_variable_rank", "means", "variances", "variances_norm"]
].head()


In [ ]:
sc.pl.highly_variable_genes(adata)

## Dimensionality reduction

In [ ]:
sc.tl.pca(adata, n_comps=100, use_highly_variable=True) # run on normalized data

In [ ]:
plt.plot(
    range(len(adata.uns["pca"]["variance_ratio"])),
    np.cumsum(adata.uns["pca"]["variance_ratio"]) * 100,
    ".-",
)
plt.axvline(30, color="r")
plt.xlabel("Principal Component", fontsize=14)
plt.ylabel("% Variance Explained", fontsize=14)

In [ ]:
adata.obsm["X_pca"] = adata.obsm["X_pca"][:, 0:30]

In [ ]:
adata.obsm["X_pca"].shape

In [ ]:
sc.pl.pca_scatter(adata, color="total_counts")

In [ ]:
# use the loadings to identify the most influential genes onto each of the PCA.
df_loadings = pd.DataFrame(
    adata.varm["PCs"][:, :30],
    index=adata.var.index,
    columns=["PC-" + str(j) for j in range(30)],
)

In [ ]:
df_loadings_hvg = df_loadings.loc[adata.var_names[adata.var["highly_variable"]]]
df_loadings_hvg

In [ ]:
np.abs(df_loadings_hvg).sort_values(by="PC-0", ascending=False)["PC-0"] # absolute value

## Nearest neighbor graph

In [ ]:
sc.pp.neighbors(
    adata, n_neighbors=30, use_rep="X_pca", metric="euclidean", key_added="neighbors_30"
)

In [ ]:
adata.obsp["neighbors_30_distances"]

In [ ]:
adata.obsp["neighbors_30_connectivities"]

## UMAP visualization

In [ ]:
sc.tl.umap(adata, neighbors_key="neighbors_30", min_dist=0.1)

In [ ]:
adata.obsm["X_umap"]

In [ ]:
adata.obsm["X_umap"].shape

In [ ]:
#sc.pl.umap(adata)

In [ ]:
sc.pl.umap(adata, color="total_counts")

In [ ]:
sc.tl.tsne(adata, use_rep="X_pca")
sc.pl.tsne(adata, color="total_counts")

# Downstream analysis

## Clustering

In [ ]:
sc.tl.leiden(
    adata,
    resolution=0.25,
    random_state=0,
    neighbors_key="neighbors_30",
    key_added="leiden_r0.25",
)
sc.tl.leiden(
    adata,
    resolution=0.5,
    random_state=0,
    neighbors_key="neighbors_30",
    key_added="leiden_r0.5",
)
sc.tl.leiden(
    adata,
    resolution=1,
    random_state=0,
    neighbors_key="neighbors_30",
    key_added="leiden_r1",
)

In [ ]:
#pd.crosstab(adata.obs["leiden_r0.25"], adata.obs["leiden_r0.5"], adata.obs["leiden_r1"])

In [ ]:
sc.pl.umap(
    adata,
    color=["leiden_r0.25", "leiden_r0.5", "leiden_r1"],
    legend_loc="on data",
)

In [ ]:
adata

In [ ]:
#Inspecting quality control metrics
df_temp = pd.DataFrame(
    {
        "leiden_0.25": adata.obs["leiden_r0.25"],
        "leiden_0.5": adata.obs["leiden_r0.5"],
        "leiden_1": adata.obs["leiden_r1"],
        "%-Mito": adata.obs["pct_counts_mt"],
        "%-Ribo": adata.obs["pct_counts_ribo"],
        "library_size": adata.obs["log1p_total_counts"],
        "n_genes_per_cell": adata.obs["log1p_n_genes_by_counts"],
    },
    index=adata.obs.index,
)
df_temp

In [ ]:
fig = plt.figure(figsize=(8 * 2, 6 * 3))
ax = fig.add_subplot(3, 1, 1)
sb.boxplot(
    x="leiden_1",
    y="%-Mito",
    hue="leiden_1",
    data=df_temp,
    ax=ax,
    palette=sc.pl.palettes.zeileis_28,
)
ax.grid("on")
ax.legend().set_visible(False)

ax = fig.add_subplot(3, 1, 2)
sb.boxplot(
    x="leiden_1",
    y="library_size",
    hue="leiden_1",
    data=df_temp,
    ax=ax,
    palette=sc.pl.palettes.zeileis_28,
)
ax.grid("on")
ax.legend().set_visible(False)

ax = fig.add_subplot(3, 1, 3)
sb.boxplot(
    x="leiden_1",
    y="n_genes_per_cell",
    hue="leiden_1",
    data=df_temp,
    ax=ax,
    palette=sc.pl.palettes.zeileis_28,
)
ax.grid("on")
ax.legend(loc="lower left", ncol=5)

In [ ]:
fig = plt.figure(figsize=(8 * 2, 6 * 3))
ax = fig.add_subplot(3, 1, 1)
sb.boxplot(
    x="leiden_0.5",
    y="%-Mito",
    hue="leiden_0.5",
    data=df_temp,
    ax=ax,
    palette=sc.pl.palettes.zeileis_28,
)
ax.grid("on")
ax.legend().set_visible(False)

ax = fig.add_subplot(3, 1, 2)
sb.boxplot(
    x="leiden_0.5",
    y="library_size",
    hue="leiden_0.5",
    data=df_temp,
    ax=ax,
    palette=sc.pl.palettes.zeileis_28,
)
ax.grid("on")
ax.legend().set_visible(False)

ax = fig.add_subplot(3, 1, 3)
sb.boxplot(
    x="leiden_0.5",
    y="n_genes_per_cell",
    hue="leiden_0.5",
    data=df_temp,
    ax=ax,
    palette=sc.pl.palettes.zeileis_28,
)
ax.grid("on")
ax.legend(loc="lower left", ncol=5)

In [ ]:
# Inspecting quality control metrics
sc.pl.umap(
    adata,
    color=["total_counts", "pct_counts_mt", "pct_counts_ribo", "doublet_score", "doublet"],
)

###clusters_interest = [""]  # explore
colors = ["red"]
plt.scatter(adata.obsm["X_umap"][:, 0], adata.obsm["X_umap"][:, 1], s=1, color="gray")
for counter, item in enumerate(clusters_interest):
    cells_in_cluster = adata.obs["leiden_r1.5"] == item
    plt.scatter(
        adata.obsm["X_umap"][cells_in_cluster, 0],
        adata.obsm["X_umap"][cells_in_cluster, 1],
        s=1,
        color=colors[counter],
        label="Cluster-" + str(item),
    )

plt.legend(markerscale=5)
plt.axis("off")

In [ ]:
#adata_clean = adata[~np.isin(adata.obs["leiden_r1.5"], clusters_interest), :]

## Cluster annotation

### Mannual annotation

#### Marker gene predictions

In [ ]:
# Calculate marker genes
sc.tl.rank_genes_groups(
    adata, groupby="leiden_r0.5", method="wilcoxon", key_added="dea_leiden_0.5"
)

In [ ]:
# Plot marker genes
sc.pl.rank_genes_groups(adata, key="dea_leiden_0.5", fontsize=12)

In [ ]:
df_wt = sc.get.rank_genes_groups_df(adata, group=str(0), key="dea_leiden_0.5")
df_wt

In [ ]:
for cluster in np.unique(adata.obs["leiden_r0.5"]):
    df = sc.get.rank_genes_groups_df(adata, group=str(cluster), key="dea_leiden_0.5")
    df.to_csv(data_path + "wilcoxon_cluster_" + str(cluster) + "_vs_rest.csv")

In [ ]:
sc.pl.rank_genes_groups_dotplot(
    adata, groupby="leiden_r0.5", standard_scale="var", n_genes=5, key="dea_leiden_0.5"
)

In [ ]:
sc.tl.filter_rank_genes_groups(
    adata,
    min_in_group_fraction=0.2,
    max_out_group_fraction=0.2,
    key="dea_leiden_0.5",
    key_added="dea_leiden_0.5_filtered",
)

In [ ]:
sc.pl.rank_genes_groups_dotplot(
    adata,
    groupby="leiden_r0.5",
    standard_scale="var",
    n_genes=5,
    key="dea_leiden_0.5_filtered",
)

In [ ]:
sc.pl.umap(
    adata,
    color=["PIGR", "C3", "MUC1", "AGR2","MUC16", "leiden_r0.5"], # cluster 7
    vmax="p99",
    legend_loc="on data",
    frameon=False,
    cmap="Reds",
)

In [ ]:
sc.pl.rank_genes_groups(adata, key="dea_leiden_0.5_filtered", n_genes=10, sharey=False) # top markers for each cluster

In [ ]:
# Extract top 10 genes per group
top_genes = [adata.uns['dea_leiden_0.5']['names'][group][:10] for group in adata.uns['dea_leiden_0.5']['names'].dtype.names]

# Print the gene names
for group, genes in zip(adata.uns['dea_leiden_0.5']['names'].dtype.names, top_genes):
    print(f"Group: {group} → Top Genes: {', '.join(genes)}")


#### Assigning cell type

In [ ]:
cl_annotation = {
    "0": "T lymphocytes",
    "1": "Epithelial cells",
    "2": "Cytotoxic (CD8+) T cells",
    "3": "Macrophage/antigen presenting cells",
    "4": "Natural killer (NK) cells",
    "5": "Fibroblasts",
    "6": "Epithelial cells",
    "7": "Epithelial cells",
    "8": "Epithelial cells",
    "9": "Mast cells",
    "10": "Endothelial cells",
    "11": "Plasma cells",
    "12": "Epithelial cells",
    "13": "Epithelial cells",
    "14": "Smooth muscle cells",
    "15": "Plasma cells",
    "16": "Dendritic cells",
    "17": "Neutrophils",
    "18": "Epithelial cells"
}

adata.obs["manual_celltype_annotation"] = adata.obs["leiden_r0.5"].map(cl_annotation)


In [ ]:
sc.pl.umap(
    adata,
    color="manual_celltype_annotation",
    legend_loc="right margin", 
    cmap="viridis"
)

##### Sub-clustering of epithelial cell clusters

In [ ]:
epithelial = adata[adata.obs['manual_celltype_annotation'] =="Epithelial cells"].copy()

In [ ]:
sc.pp.normalize_total(epithelial)
sc.pp.log1p(epithelial)
sc.pp.highly_variable_genes(epithelial, n_top_genes=2000)
epithelial=epithelial[:, epithelial.var['highly_variable']]
sc.pp.scale(epithelial)
sc.tl.pca(epithelial, n_comps=30)
sc.pp.neighbors(epithelial, n_neighbors=15, n_pcs=30)
sc.tl.pca(epithelial)

In [ ]:
# Try a few resolutions to see what gives meaningful granularity
for res in [0.3, 0.5, 0.8, 1.0]:
    sc.tl.leiden(epithelial, resolution=res, key_added=f'leiden_r{res}')
    n = epithelial.obs[f'leiden_r{res}'].nunique()
    print(f"resolution={res} → {n} clusters")

# Pick your favorite (say res=0.5)
sc.tl.leiden(epithelial, resolution=0.3, key_added='subcluster')
sc.pl.umap(epithelial, color='subcluster', legend_loc='right')

In [ ]:
# Calculate marker genes
sc.tl.rank_genes_groups(
    epithelial, groupby="subcluster", method="wilcoxon", pts=True)
sc.pl.rank_genes_groups(epithelial, n_genes=5, sharey=False)

In [ ]:
#Extract top 5 genes per group
top_genes = [epithelial.uns['rank_genes_groups']['names'][group][:5] for group in epithelial.uns['rank_genes_groups']['names'].dtype.names]

# Print the gene names
for group, genes in zip(epithelial.uns['rank_genes_groups']['names'].dtype.names, top_genes):
    print(f"Group: {group} → Top Genes: {', '.join(genes)}")


In [ ]:
subcluster_annotation = {
    "0": "Basal Epithelial",
    "1": "Secretory Epithelial",
    "2": "Proliferative Epithelial",
    "3": "Basal Epithelial",
    "4": "Squamas Differentiated Epithelial",
    "5": "Luminal Epithelial",
    "6": "Immune",
    "7": " Luminal Epithelial cells"
}

epithelial.obs["epithelial_subtype"] = epithelial.obs["subcluster"].map(subcluster_annotation)

In [ ]:
sc.pl.umap(
    epithelial,
    color="epithelial_subtype",
    legend_loc="right margin", 
    cmap="viridis"
)

### Automatic annotation

In [ ]:
adata_celltypist = adata.copy()  # make a copy of our adata
adata_celltypist.X = adata.layers["raw_data"]  # set adata.X to raw counts
sc.pp.normalize_total(adata_celltypist, target_sum=10**4)  # normalize to 10,000 counts per cell
sc.pp.log1p(adata_celltypist)  # log-transform
sc.pp.pca(adata_celltypist, n_comps=50) # at least 50 pca for celltypist
sc.pp.neighbors(adata_celltypist, n_neighbors=10, n_pcs=50)
# make .X dense instead of sparse, for compatibility with celltypist:
adata_celltypist.X = adata_celltypist.X.toarray()
import scanpy as sc

sc.pp.pca(adata_celltypist, n_comps=50)
sc.pp.neighbors(adata_celltypist, n_neighbors=10, n_pcs=50)


In [ ]:
models.download_models(
    force_update=True, model=["Immune_All_Low.pkl", "Immune_All_High.pkl", "Adult_Human_Skin.pkl", 
                              "Adult_Human_Vascular.pkl", "Adult_Human_PancreaticIslet.pkl"]
)

In [ ]:
model_immune_low = models.Model.load(model="Immune_All_Low.pkl")
model_immune_high = models.Model.load(model="Immune_All_High.pkl")
model_human_skin = models.Model.load(model="Adult_Human_Skin.pkl")
model_human_vascular = models.Model.load(model="Adult_Human_Vascular.pkl")
model_human_pancreas = models.Model.load(model="Adult_Human_PancreaticIslet.pkl")

In [ ]:
model_human_vascular.cell_types

In [ ]:
predictions_immune_high = celltypist.annotate(
    adata_celltypist, model=model_immune_high, majority_voting=True
) # run the model

predictions_immune_high_adata = predictions_immune_high.to_adata() # transform the predictions to adata to get the full output
#copy the results to our original AnnData object
adata.obs["celltypist_cell_label_immune_high"] = predictions_immune_high_adata.obs.loc[
    adata.obs.index, "majority_voting"
]
adata.obs["celltypist_conf_score_immune_high"] = predictions_immune_high_adata.obs.loc[
    adata.obs.index, "conf_score"
]

In [ ]:
predictions_immune_low = celltypist.annotate(
    adata_celltypist, model=model_immune_low, majority_voting=True
)
predictions_immune_low_adata = predictions_immune_low.to_adata()
adata.obs["celltypist_cell_label_immune_low"] = predictions_immune_low_adata.obs.loc[
    adata.obs.index, "majority_voting"
]
adata.obs["celltypist_conf_score_immune_low"] = predictions_immune_low_adata.obs.loc[
    adata.obs.index, "conf_score"
]

In [ ]:
predictions_fibroblasts = celltypist.annotate(
    adata_celltypist, model=model_human_skin, majority_voting=True
)
predictions_fibroblasts_adata = predictions_fibroblasts.to_adata()
adata.obs["celltypist_cell_label_fibroblasts"] = predictions_fibroblasts_adata.obs.loc[
    adata.obs.index, "majority_voting"
]
adata.obs["celltypist_conf_score_fibroblasts"] = predictions_fibroblasts_adata.obs.loc[
    adata.obs.index, "conf_score"
]

In [ ]:
predictions_endothelial = celltypist.annotate(
    adata_celltypist, model=model_human_vascular, majority_voting=True
)
predictions_endothelial_adata = predictions_endothelial.to_adata()
adata.obs["celltypist_cell_label_endothelial"] = predictions_endothelial_adata.obs.loc[
    adata.obs.index, "majority_voting"
]
adata.obs["celltypist_conf_score_endothelial"] = predictions_endothelial_adata.obs.loc[
    adata.obs.index, "conf_score"
]

In [ ]:
predictions_epithelial = celltypist.annotate(
    adata_celltypist, model=model_human_pancreas, majority_voting=True
)
predictions_epithelial_adata = predictions_epithelial.to_adata()
adata.obs["celltypist_cell_label_epithelial"] = predictions_epithelial_adata.obs.loc[
    adata.obs.index, "majority_voting"
]
adata.obs["celltypist_conf_score_epithelial"] = predictions_epithelial_adata.obs.loc[
    adata.obs.index, "conf_score"
]

In [ ]:
sc.pl.umap(
    adata,
    color=["celltypist_cell_label_immune_high", "celltypist_conf_score_immune_high"],
    frameon=False,
    sort_order=False,
    wspace=0.5,
)

In [ ]:
sc.pl.umap(
    adata,
    color=["celltypist_cell_label_immune_low", "celltypist_conf_score_immune_low"],
    frameon=False,
    sort_order=False,
    wspace=0.5,
)

In [ ]:
sc.pl.umap(
    adata,
    color=["celltypist_cell_label_fibroblasts", "celltypist_conf_score_fibroblasts"],
    frameon=False,
    sort_order=False,
    wspace=0.5,
)

In [ ]:
sc.pl.umap(
    adata,
    color=["celltypist_cell_label_endothelial", "celltypist_conf_score_endothelial"],
    frameon=False,
    sort_order=False,
    wspace=0.5,
)

In [ ]:
sc.pl.umap(
    adata,
    color=["celltypist_cell_label_epithelial", "celltypist_conf_score_epithelial"],
    frameon=False,
    sort_order=False,
    wspace=0.5,
)

In [ ]:
sc.pl.umap(adata, color=["celltypist_cell_label_immune_high", "celltypist_cell_label_immune_low",
                         "celltypist_cell_label_fibroblasts","celltypist_cell_label_endothelial",
                         "celltypist_cell_label_epithelial"],
                         frameon=False)

In [ ]:
adata.obs["combined_annotation"] = adata.obs[
    ["celltypist_cell_label_immune_high", "celltypist_cell_label_immune_low",
     "celltypist_cell_label_fibroblasts", "celltypist_cell_label_endothelial",
     "celltypist_cell_label_epithelial"]
].mode(axis=1).iloc[:, 0]  # Select first mode if multiple exist


In [ ]:
sc.pl.umap(
    adata,
    color="combined_annotation",
    legend_loc="right margin",
    cmap="viridis"
)

## Trajectory inference

### Pre-processing

In [ ]:
#subset epithelial cells
epithelial_cells = adata[adata.obs['manual_celltype_annotation'] =="Epithelial cells"].copy()
sc.pp.filter_genes(epithelial_cells, min_counts=10)

In [ ]:
sc.pp.normalize_total(epithelial_cells)
sc.pp.log1p(epithelial_cells)
sc.pp.highly_variable_genes(epithelial_cells, n_top_genes=2000)
sc.pp.scale(epithelial_cells)
sc.tl.pca(epithelial_cells, n_comps=50)
#sc.tl.umap(epithelial_cells)

### Diffusion maps

In [ ]:
#palantir.utils.run_diffusion_maps(epithelial_cells, n_components=30)
palantir.utils.run_diffusion_maps(epithelial, n_components=30)

In [ ]:
#palantir.utils.determine_multiscale_space(epithelial_cells)
palantir.utils.determine_multiscale_space(epithelial)

### Selecting early state gene

In [ ]:
early_genes = ['KRT5', 'KLF5', 'TP63']
tumor_genes = ['MKI67', 'TOP2A', 'CDKN2A', 'S100A7']

In [ ]:
#sc.tl.score_genes(epithelial_cells, gene_list=early_genes, score_name='early_score')
#sc.tl.score_genes(epithelial_cells, gene_list=tumor_genes, score_name='tumor_score')
#sc.pl.umap(epithelial_cells, color=['early_score', 'tumor_score'], cmap='viridis')

sc.tl.score_genes(epithelial, gene_list=early_genes, score_name='early_score')
sc.tl.score_genes(epithelial, gene_list=tumor_genes, score_name='tumor_score')
sc.pl.umap(epithelial, color=['early_score', 'tumor_score'], cmap='viridis')

In [ ]:
# Find cells with high early score and low tumor score
#list = epithelial_cells.obs[['early_score', 'tumor_score']]
#candidate_cells = list[(list['early_score'] > 0.5) & (list['tumor_score'] < 0.1)]
# Pick the cell with highest early score
#start_cell = candidate_cells['early_score'].idxmax()
#print("Start cell ID:",start_cell)



# Find cells with high early score and low tumor score
list = epithelial.obs[['early_score', 'tumor_score']]
candidate_cells = list[(list['early_score'] > 0.5) & (list['tumor_score'] < 0.1)]
# Pick the cell with highest early score
start_cell = candidate_cells['early_score'].idxmax()
print("Start cell ID:",start_cell)

### Running plantir

In [ ]:
#pr_res = palantir.core.run_palantir(epithelial_cells, start_cell, num_waypoints=500)
pr_res = palantir.core.run_palantir(epithelial, start_cell, num_waypoints=500)

In [ ]:
#epithelial_cells.obs["palantir_pseudotime"]
epithelial.obs["palantir_pseudotime"]

In [ ]:
# Get the branch probabilities
# As shown below, palantir used the 6 terminally differentiated states
#epithelial_cells.obsm["palantir_fate_probabilities"]

# Get the branch probabilities
# As shown below, palantir used the 6 terminally differentiated states
epithelial.obsm["palantir_fate_probabilities"]

In [ ]:
# You can rename the different branches:
#epithelial_cells.obsm["palantir_fate_probabilities"].columns = [
   # "Branch1",
    #"Branch2",
  #  "Branch3",
   # "Branch4"]

# You can rename the different branches:
epithelial.obsm["palantir_fate_probabilities"].columns = ["Branch1","Branch2"]

In [ ]:
# Get the entropy
#epithelial_cells.obs["palantir_entropy"]

# Get the entropy
epithelial.obs["palantir_entropy"]

In [ ]:
palantir.plot.plot_palantir_results(epithelial_cells, s=3)
plt.show()

In [ ]:
# Visualize branch probability
nrow = 2
ncol = 3
fig = plt.figure(figsize=(8 * ncol, 6 * nrow))
for j, item in enumerate(pr_res.branch_probs.columns):
    ax = fig.add_subplot(nrow, ncol, j + 1)
    im1 = ax.scatter(
        epithelial_cells.obsm["X_umap"][:, 0],
        epithelial_cells.obsm["X_umap"][:, 1],
        c=epithelial_cells.obsm["palantir_fate_probabilities"][item],
        s=2,
        cmap="plasma",
    )
    ax.axis("off")
    ax.set_title("Branch probability - " + str(j + 1), fontsize=16)
    fig.colorbar(im1)

In [ ]:
example_cells = ["ATCGGCGTCCATAAGC-1", "CCCGAAGTCGACGAGA-1", "CTCTGGTGTAAGATAC-1", "TCACGGGTCATTGCTT-1"]
fig = plt.figure(figsize=(8 * 3, 6 * 2))  # Wider for 3 columns

# Subplot 1: UMAP with highlighted cells (spans 2 columns)
ax = fig.add_subplot(2, 3, (1, 2))
cell_id = [epithelial_cells.obs_names.get_loc(k) for k in example_cells]
ax.scatter(epithelial_cells.obsm["X_umap"][:, 0], epithelial_cells.obsm["X_umap"][:, 1], s=1, c='lightgray', alpha=0.5)

# Highlight example cells with different colors
colors = ['r', 'g', 'b', 'orange']
for i, (cell, color) in enumerate(zip(example_cells, colors)):
    ax.scatter(
        epithelial_cells.obsm["X_umap"][cell_id[i], 0],
        epithelial_cells.obsm["X_umap"][cell_id[i], 1],
        s=100,
        c=color,
        label=cell,
    )
ax.legend()
ax.axis("off")
ax.set_title("UMAP with example cells")

# Individual fate probability plots for each cell
subplot_positions = [3, 4, 5, 6]  # Top right, then bottom row
for i, cell in enumerate(example_cells):
    ax = fig.add_subplot(2, 3, subplot_positions[i])
    ax.bar(
        epithelial_cells.obsm["palantir_fate_probabilities"].columns,
        epithelial_cells.obsm["palantir_fate_probabilities"].loc[cell],
    )
    ax.set_xlabel("Branches")
    ax.set_ylabel("Probability")
    ax.set_title(cell)

plt.tight_layout()

### Gene expression trend along branch

In [ ]:
imputed_X = palantir.utils.run_magic_imputation(epithelial_cells)

In [ ]:
masks = palantir.presults.select_branch_cells(epithelial_cells, eps=0)

In [ ]:
epithelial_cells.obsm["branch_masks"]

In [ ]:
palantir.plot.plot_branch_selection(epithelial_cells)
plt.show()

In [ ]:
gene_trends = palantir.presults.compute_gene_trends(epithelial_cells)

In [ ]:
palantir.plot.plot_gene_trend_heatmaps(epithelial_cells, genes)
plt.show()

In [ ]:
palantir.plot.plot_trajectories(epithelial_cells, pseudotime_interval=(0, .9))
# When using cell_color="branch_selection" be aware of the overlap between branches:
palantir.plot.plot_trajectories(epithelial_cells, cell_color = "branch_selection", pseudotime_interval=(0, .9))
plt.show()

### Cell Rank

In [ ]:
import cellrank as cr

In [ ]:
from cellrank.kernels import PseudotimeKernel
epithelial.obsp['neighbors_connectivities'] = epithelial.obsp['neighbors_30_connectivities']
pk = PseudotimeKernel(epithelial, time_key="palantir_pseudotime")

In [ ]:
pk

In [ ]:
pk.compute_transition_matrix()

In [ ]:
pk.plot_random_walks(
    seed=0,
    n_sims=100,
    start_ixs={"leiden_r0.5": "1"},  # Replace "0" with the desired cluster label for your analysis.
    basis="umap",                   # Use "umap" or "tsne" depending on what you generated.
    legend_loc="right",
    dpi=150,
)

In [ ]:
from cellrank.kernels import ConnectivityKernel

ck = ConnectivityKernel(epithelial).compute_transition_matrix()

In [ ]:
ck

In [ ]:
combined_kernel = 0.8 * pk + 0.2 * ck
combined_kernel

In [ ]:
from cellrank.estimators import GPCCA

g = GPCCA(pk)
print(g)

In [ ]:
g.fit(n_states=10, cluster_key="subcluster")
g.plot_macrostates(which="all")

In [ ]:
g.predict_terminal_states(method="top_n", n_states=6)
g.plot_macrostates(which="terminal")

In [ ]:
g.compute_fate_probabilities()
g.plot_fate_probabilities(legend_loc="right")

In [ ]:
cr.pl.circular_projection(epithelial, keys="subcluster", legend_loc="right")

In [ ]:
mono_drivers = g.compute_lineage_drivers(lineages="1_2")
mono_drivers.head(10)